In [1]:
from google.api_core import page_iterator
from google.cloud import storage

In [4]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
def _item_to_value(iterator, item):
    return item

def list_directories(bucket_name, prefix):
    if prefix and not prefix.endswith('/'):
        prefix += '/'

    extra_params = {
        "projection": "noAcl",
        "prefix": prefix,
        "delimiter": '/'
    }

    gcs = storage.Client()

    path = "/b/" + bucket_name + "/o"

    iterator = page_iterator.HTTPIterator(
        client=gcs,
        api_request=gcs._connection.api_request,
        path=path,
        items_key='prefixes',
        item_to_value=_item_to_value,
        extra_params=extra_params,
    )

    return [x for x in iterator]

In [13]:
#prefix = 'raw/18_qudini/1801'
#test with adyen that I know to be wrong 
prefix = 'raw/50_adyen/5004__ad_payment'

dir_list = list_directories('swarovski-advanced-analytics-data-ingestion-prod', prefix)
dir_list

['raw/50_adyen/5004__ad_payment/5004/',
 'raw/50_adyen/5004__ad_payment/v1/',
 'raw/50_adyen/5004__ad_payment/v2/',
 'raw/50_adyen/5004__ad_payment/v2_backfill/']

In [14]:
for item in dir_list:
  dir_list_v2 = list_directories('swarovski-advanced-analytics-data-ingestion-prod',item)
  print(dir_list_v2)

['raw/50_adyen/5004__ad_payment/5004/v1/']
['raw/50_adyen/5004__ad_payment/v1/2021/', 'raw/50_adyen/5004__ad_payment/v1/2022/', 'raw/50_adyen/5004__ad_payment/v1/v1/']
['raw/50_adyen/5004__ad_payment/v2/2022/']
['raw/50_adyen/5004__ad_payment/v2_backfill/2022/', 'raw/50_adyen/5004__ad_payment/v2_backfill/202205/', 'raw/50_adyen/5004__ad_payment/v2_backfill/202206/', 'raw/50_adyen/5004__ad_payment/v2_backfill/202207/', 'raw/50_adyen/5004__ad_payment/v2_backfill/202208/', 'raw/50_adyen/5004__ad_payment/v2_backfill/202209/']


In [28]:
#the function works, we can now implement the check that for each string checks whether the correct path is present 
#the expected path for each subfolder is the following 
# data_type/source_system/pipeline_code/version/year

#first we define bucket and pipeline to check

bucket = 'swarovski-advanced-analytics-data-ingestion-prod'
prefix = 'raw/50_adyen/5004__ad_payment/v1/' 

dir_list = list_directories(bucket,prefix)
dir_list

['raw/50_adyen/5004__ad_payment/v1/2021/',
 'raw/50_adyen/5004__ad_payment/v1/2022/',
 'raw/50_adyen/5004__ad_payment/v1/v1/']

In [32]:
for dir_ in dir_list: 

  dir_split = dir_.split('/')
  print(dir_split[-2])
  if dir_split[-2] in ('2020','2021','2022'):
    is_policy_respected = True
  else: is_policy_respected = False
  print(is_policy_respected)

2021
True
2022
True
v1
False
